In [1]:
import os
import numpy as np
import math
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import dlib

image1_path = os.getenv('IMAGE1_PATH')
image2_path = os.getenv('IMAGE2_PATH')
output_path = os.getenv('OUTPUT_PATH')

In [ ]:
def get_landmarks(image, detector, predictor):
        # Detect faces in the image
        detected_faces = detector(image, 1)
        if not detected_faces:
            raise ValueError("No faces detected in the image.")
        # Get the landmarks/parts for the face
        shape = predictor(image, detected_faces[0])
        return np.array([[part.x, part.y] for part in shape.parts()])

def warp_image(image, points_src, points_dst, shape):
        # Compute the transformation matrix
        matrix = cv2.estimateAffinePartial2D(points_src, points_dst)[0]
        # Warp the image using the matrix
        warped_image = cv2.warpAffine(image, matrix, shape, flags=cv2.INTER_LINEAR)
        return warped_image

def blend_images(image1, image2, alpha=0.5):
        # Weighted average of the two images
        return cv2.addWeighted(image1, alpha, image2, (1 - alpha), 0)

In [ ]:
def process_images(image1_path, image2_path, output_path):
    
    damaged_face = cv2.imread(image1_path)
    complete_face = cv2.imread(image2_path)

    # Ensure both images were loaded correctly
    if damaged_face is None or complete_face is None:
        raise ValueError("Could not load one or both images. Please check the file paths.")

    # Get the dimensions to resize to (the smallest of the two images)
    new_height = min(damaged_face.shape[0], complete_face.shape[0])
    new_width = min(damaged_face.shape[1], complete_face.shape[1])

    # Resize both images to the smallest dimensions
    image1 = cv2.resize(damaged_face, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    image2 = cv2.resize(complete_face, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    
    # Get landmarks for both images
    landmarks1 = get_landmarks(image1, detector, predictor)
    landmarks2 = get_landmarks(image2, detector, predictor)

    # Compute average landmarks
    average_landmarks = (landmarks1 + landmarks2) / 2

    # Warp both images towards the average landmarks
    warped_image1 = warp_image(image1, landmarks1, average_landmarks, image1.shape[:2][::-1])
    warped_image2 = warp_image(image2, landmarks2, average_landmarks, image2.shape[:2][::-1])

    # Perform the blending to create the morphed image
    output = blend_images(warped_image1, warped_image2, alpha=0.5)
    
    cv2.imwrite(output_path, output)
    
    # Optionally, you can return the path to the output image
    return output_path


# Execute the processing function
process_images(image1_path, image2_path, output_path)